In [1]:
import nltk

In [2]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [3]:
from nltk.corpus import movie_reviews

In [4]:
negids = movie_reviews.fileids('neg')

In [5]:
posids = movie_reviews.fileids('pos')

In [6]:
len(negids), len(posids)

(1000, 1000)

In [7]:
negfeats = [movie_reviews.words(fileids=[x]) for x in negids]

In [8]:
posfeats = [movie_reviews.words(fileids=[x]) for x in posids]

In [9]:
f'Общее количество отзывов: {len(posfeats) + len(negfeats)}'

'Общее количество отзывов: 2000'

In [10]:
f'Доля позитивных отзывов: {len(posfeats) / (len(posfeats) + len(negfeats))}'

'Доля позитивных отзывов: 0.5'

In [11]:
import numpy as np

In [74]:
y = np.array([0] * 1000 + [1] * 1000)

In [75]:
y.shape

(2000,)

In [14]:
allfeats = []

In [15]:
for i in negfeats:
  allfeats.append(' '.join(i))

for j in posfeats:
  allfeats.append(' '.join(j))

In [16]:
len(allfeats)

2000

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
from sklearn.linear_model import LogisticRegression

In [23]:
from sklearn.pipeline import Pipeline

In [26]:
pipe1 = Pipeline([('vectoriser', CountVectorizer()), ('logreg', LogisticRegression())])

In [27]:
pipe2 = Pipeline([('vectoriser', TfidfVectorizer()), ('logreg', LogisticRegression())])

In [29]:
from sklearn.model_selection import cross_val_score

In [30]:
answer1_1 = cross_val_score(pipe1, allfeats, y, cv=5)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [32]:
answer1_2 = cross_val_score(pipe2, allfeats, y, cv=5)

In [33]:
answer1_1, answer1_2

(array([0.805 , 0.845 , 0.84  , 0.8725, 0.85  ]),
 array([0.82  , 0.825 , 0.825 , 0.815 , 0.8175]))

In [46]:
print('Ответы на вопрос 1:')
round(answer1_1.mean(), 2), round(answer1_1.std(), 2), round(answer1_2.mean(), 2), round(answer1_2.std(), 3)

Ответы на вопрос 1:


(0.84, 0.02, 0.82, 0.004)

In [49]:
answers_2 = []

In [ ]:
for i in [1, 10, 20, 50, 100]:
  temp_pipe = Pipeline([('vectoriser', CountVectorizer(min_df=i)), ('logreg', LogisticRegression())])
  accuracy_list = cross_val_score(temp_pipe, allfeats, y, cv=5)
  answers_2.append(accuracy_list.mean())

In [53]:
print('Ответы на вопрос 2:')
for i, y in zip([1, 10, 20, 50, 100], answers_2):
  print(f'{i}: {round(y, 3)}')

Ответы на вопрос 2:
1: 0.842
10: 0.84
20: 0.832
50: 0.814
100: 0.788


In [54]:
from sklearn.svm import LinearSVC

In [55]:
from sklearn.linear_model import SGDClassifier

In [88]:
pipe_logreg = Pipeline([('vectoriser', CountVectorizer()), ('logreg', LogisticRegression(random_state=42))])

In [89]:
pipe_svc = Pipeline([('vectoriser', CountVectorizer()), ('linearSVC', LinearSVC(random_state=42))])

In [90]:
pipe_cgd = Pipeline([('vectoriser', CountVectorizer()), ('SGDClassifier', SGDClassifier(random_state=42))])

In [102]:
non_default_sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=5, 
                                shuffle=True, verbose=0, epsilon=0.1, n_jobs=1, random_state=42, learning_rate='optimal', 
                                eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False)

In [104]:
pipe_non_default_cgd = Pipeline([('vectoriser', CountVectorizer()), ('SGDClassifier', non_default_sgd)])

In [109]:
answers_3 = []

In [ ]:
for p in [pipe_logreg, pipe_svc, pipe_cgd, pipe_non_default_cgd]:
  answers_3.append( cross_val_score(p, allfeats, y, cv=5).mean() )

In [113]:
print('Ответы на вопрос 3:')
for i, j in zip(['logreg', 'svc', 'sgd', 'sgd_non_default'], answers_3):
  print(f'{str(i)}: {round(j, 2)}')

Ответы на вопрос 3:
logreg: 0.84
svc: 0.83
sgd: 0.84
sgd_non_default: 0.78


In [116]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [120]:
stopwords_nltk = nltk.corpus.stopwords.words('english')

In [125]:
len(stopwords_nltk)

179

In [126]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

In [130]:
stopwords_sklearn = list(ENGLISH_STOP_WORDS)

In [131]:
len(stopwords_sklearn)

318

In [132]:
pipe_stop_nltk = Pipeline([('vectoriser', CountVectorizer(stop_words=stopwords_nltk)), ('logreg', LogisticRegression())])

In [133]:
pipe_stop_sklearn = Pipeline([('vectoriser', CountVectorizer(stop_words=stopwords_sklearn)), ('logreg', LogisticRegression())])

In [135]:
answer4_1 = cross_val_score(pipe_stop_nltk, allfeats, y, cv=5).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [137]:
answer4_2 = cross_val_score(pipe_stop_sklearn, allfeats, y, cv=5).mean()

In [138]:
print(f'Ответы на вопрос 4:')
answer4_1, answer4_2

Ответы на вопрос 4:


(0.8400000000000001, 0.8365)

In [149]:
bigrams = list(nltk.bigrams(stopwords_nltk))

In [150]:
bigrams[:5]

[('i', 'me'), ('me', 'my'), ('my', 'myself'), ('myself', 'we'), ('we', 'our')]

In [151]:
stopwords_nltk[:5]

['i', 'me', 'my', 'myself', 'we']

In [158]:
united_list = []
united_list.extend(stopwords_nltk)

In [160]:
for i in bigrams:
  united_list.append(' '.join(i))

In [162]:
len(united_list)

357

In [163]:
pipe_united_list = Pipeline([('vectoriser', CountVectorizer(stop_words=united_list)), ('logreg', LogisticRegression())])

In [164]:
answer5_1 = cross_val_score(pipe_united_list, allfeats, y, cv=5).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [168]:
pipe_un_lettergrams = Pipeline([('vectoriser', CountVectorizer(stop_words=united_list, ngram_range=(3, 5), analyzer='char_wb')), 
                                ('logreg', LogisticRegression())])

In [169]:
answer5_2 = cross_val_score(pipe_un_lettergrams, allfeats, y, cv=5).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [171]:
print(f'Ответы на вопрос 5:')
answer5_1, answer5_2

Ответы на вопрос 5:


(0.8400000000000001, 0.826)